In [181]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [182]:
data = pd.read_csv('../data/training_data.csv')
data['month'] = data['month'].astype('category')
data['day_of_week'] = data['day_of_week'].astype('category')
# data["hour_of_day"] = data["hour_of_day"].astype('category')
data["holiday"] = data["holiday"].astype('category')
data["weekday"] = data["weekday"].astype('category')
data["summertime"] = data["summertime"].astype('category')
data['increase_stock'] = data['increase_stock'].astype('category')
data['is_high_demand'] = data['increase_stock'].apply(lambda entity: 1 if entity == 'high_bike_demand' else 0)
data['is_high_demand'] = data['is_high_demand'].astype('int')

data["daytime"] = data.hour_of_day.apply(lambda h: 1 if h in range(7, 20) else 0)
data["rushhour"] = data.hour_of_day.apply(lambda h: 1 if h in range(15, 20) else 0)

data = data.drop('snow', axis=1) #no information in this column
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   hour_of_day     1600 non-null   int64   
 1   day_of_week     1600 non-null   category
 2   month           1600 non-null   category
 3   holiday         1600 non-null   category
 4   weekday         1600 non-null   category
 5   summertime      1600 non-null   int64   
 6   temp            1600 non-null   float64 
 7   dew             1600 non-null   float64 
 8   humidity        1600 non-null   float64 
 9   precip          1600 non-null   float64 
 10  snowdepth       1600 non-null   float64 
 11  windspeed       1600 non-null   float64 
 12  cloudcover      1600 non-null   float64 
 13  visibility      1600 non-null   float64 
 14  increase_stock  1600 non-null   category
 15  is_high_demand  1600 non-null   int64   
 16  daytime         1600 non-null   int64   
 17  rushhour      

In [183]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
y = data['is_high_demand']
data_weather = data.drop(
    columns=['is_high_demand', 'increase_stock', 'hour_of_day', 'day_of_week', 'month', 'holiday', 'weekday','daytime','rushhour'])
X = scaler.fit_transform(data_weather)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=123)

weather_model = LinearRegression()
weather_model.fit(X_train, y_train)
print(pd.DataFrame(X).describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   summertime  1600 non-null   int64  
 1   temp        1600 non-null   float64
 2   dew         1600 non-null   float64
 3   humidity    1600 non-null   float64
 4   precip      1600 non-null   float64
 5   snowdepth   1600 non-null   float64
 6   windspeed   1600 non-null   float64
 7   cloudcover  1600 non-null   float64
 8   visibility  1600 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 112.6 KB
None


In [184]:
print(f'ceofs')
print(weather_model.intercept_)
print(weather_model.coef_)

ceofs
-0.8138192466580598
[-5.56042822e-04  6.31851025e-02 -5.21309866e-02  6.49687899e-03
 -1.10383706e-02  6.86816015e-03 -5.95565495e-05  7.78956846e-04
 -1.72291318e-03]
-0.0017229131847425094


In [185]:
predictions = weather_model.predict(X_test)
pd.DataFrame(predictions).describe()

-0.1355069622538172


In [190]:
errors = y_test - predictions
sns.histplot(errors)

array([-0.13550696,  0.440059  ,  0.12344861,  0.21471269])

In [ ]:
data['weather_score'] = weather_model.predict(X)
data.describe()

In [ ]:

data_subset = data[['is_high_demand','temp','weather_score','hour_of_day','rushhour','daytime']]
covariates = data_subset.columns[data_subset.columns != 'is_high_demand']
plt.figure(figsize=(10, 10), dpi=500)
sns.set(style="ticks")
pp = sns.pairplot(data_subset,
                  vars=covariates,
                  hue='is_high_demand',
                  markers=["o", "s"],
                  plot_kws={"s":8},
                  palette="viridis")
pp.map_lower(sns.kdeplot, fill=True)
pp.map_diag(sns.histplot, kde=True)

In [ ]:
corr_matrix = data_subset.corr()
cmap = sns.color_palette('viridis', as_cmap=True)
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
# Draw the heatmap with the mask and aspect ratio
sns.heatmap(corr_matrix, 
            mask=mask, 
            cmap=cmap, 
            vmax=.3,
            center=0,
            square=True, 
            linewidths=.5, 
            cbar_kws={"shrink": .5}, annot=True)

In [ ]:
sns.violinplot(data=data_subset,x='daytime',y='temp', hue='is_high_demand', palette='viridis', linewidth=.8)